In [1]:
import requests
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

### 1. Articles scrapping

In [39]:
with open('../.api_auth') as f:
    (api_login, api_pass) = [line.rstrip() for line in f]

ids = []
leads = []
texts = []

date_range = pd.date_range(start="2023-01-01", end="2023-01-31").strftime('%d.%m.%Y').to_list()
for date in tqdm(date_range):
    r1 = requests.get(f'https://api.sta.si/news/sl/{date}', auth=(api_login, api_pass))
    articles_id = eval(r1.content)

    for id in articles_id:
        r2 = requests.get(f'https://api.sta.si/news/sta/{id}', auth=(api_login, api_pass))
        ids.append(id)
        leads.append(r2.json().get('lede', ''))
        texts.append(r2.json().get('text', ''))

dataset = pd.DataFrame({'id': ids, 'lead': leads, 'text': texts})
dataset.to_json(f'../data/raw_articles/01_2023.jsonl', lines=True, orient='records')

100%|██████████| 31/31 [11:34<00:00, 22.41s/it]


### 2. Preparing evaluation dataset

In [12]:
# df = pd.read_json("..\\data\\raw_articles\\all_2023.jsonl", lines=True)
# df_eval = df.sample(200).reset_index(drop=True)
# df_eval.to_json(f'../data/raw_articles/all_2023_eval.jsonl', lines=True, orient='records')

In [10]:
df_eval = pd.read_json("..\\data\\raw_articles\\all_2023_eval.jsonl", lines=True)
df_eval.to_json(f'../data/raw_articles/all_2023_eval.jsonl', lines=True, orient='records', force_ascii=False)

In [3]:
df_eval

,id,lead,text
0,3163048,Vreme in temperature po Evropi ob 20. uri.,<pre>\nOpazovalna postaja Vreme Te...
1,3155939,,
2,3131650,Občina Vojnik je ta mesec s celjskim podjetjem...,Kot je za STA povedal župan Vojnika Branko Pet...
3,3160676,Glasbenik Balladero s pravim imenom Dominik Ba...,"Balladero, znan po denimo skladbi Zlati časi, ..."
4,3146215,Nogometaši CB24 Tabora Sežane in Brava so se v...,"* Stadion Rajka Štolfe, sodniki: Halilović, Me..."
...,...,...,...
195,3158391,Borze na stari celini danes zaradi praznika mi...,Osrednji indeksi na evropskih borzah so četrtk...
196,3140600,"Napoved ozadij, tematskih prispevkov in pogovo...",- ozadje pred začetkom pogajanj o reformi plač...
197,3162647,Cene nafte so se tudi v današnjem azijskem trg...,"Cena teksaške lahke nafte, ki bo dobavljena ma..."
198,3145481,"V Estoniji bodo danes potekale volitve, na kat...","Estonija, ki je članica tako EU kot zveze Nato..."
